# **在Keras中使用遷移學習**

### 背景說明請見[Deep Learning For Beginners Using Transfer Learning In Keras](https://towardsdatascience.com/keras-transfer-learning-for-beginners-6c9b8b7143e)




## 前置作業：

1.   下載[aditya9898/transfer-learning](https://github.com/aditya9898/transfer-learning)

  放在 drive 的 ColabNotebooks/ 目錄底下。
2.   colab 的 Edit => Notebook Settings 進去，把 Hardware accelerator 設定成 GPU。



3.   掛載 google drive 






In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
import os
os.chdir('/content/drive/My Drive/ColabNotebooks/transfer-learning-master')#把放進去的資料夾掛載

載入相依套件：

In [0]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

導入並構建所需的模型。

In [0]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(120,activation='softmax')(x) #final layer with softmax activation

我們根據結構製作一個模型。

In [0]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

檢查我們的模型結構。

In [0]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

現在我們有了我們的模型，我們使用預先訓練的權重，並且已經在數據集上訓練了我們的模型，所以我們必須將所有權重設置為不可訓練。我們將只訓練我們之前製作的最後一層。

In [0]:
for layer in model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

現在我們將訓練數據加載到ImageDataGenerator中。

In [0]:

train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('./train/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

接下來在數據集上訓練模型。
編譯製作的模型，然後訓練模型。

In [0]:

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=10)

這樣我們將可以訓練一個模型。然後通過使用model.predict（new_image），可以將訓練後的模型用於預測新的圖像並判斷屬於哪個類別。

In [75]:
img = image.load_img('./train/cats/Z.jpg', target_size=(224, 224))#測試訓練模型
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
print (preds)

[[1. 0. 0.]]
